In [1]:
from typing import List, Dict
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
dev = 'cpu'
if torch.cuda.is_available():
    dev = 'cuda:0'
print("Running on:",dev)
device = torch.device(dev)

Running on: cpu


# Quivers and quiver representations

In [3]:
class quiver:
    def __init__(self, vertices : List, edges : List):
        self.vertices = vertices
        # Add assert to check no repeated vertices
        # E.g. assert len(set(vertices)) == len(vertices)
        
        self.edges = edges
        # Add assert to check that edges is a list of pairs
        # First entry of the pair is the sourse, second is the target
        # Source and target of each edge should be in the vertex set      
        # Separate class for edges? vertices?
    
    # Check that the quiver is acyclic
    def check_acyclic(self):
        None
        # One way: find all sources, do depth-first search
        
    # Check that the vertices are in topological order    
    def check_top_order(self):
        indices = {}
        for i,v in enumerate(self.vertices):
            indices[v] = i
        for e in self.edges:
            if indices[e[0]] > indices[e[1]]:
                return False
        return True
        
    # Get the incoming edges for every vertex
    # Incoming vertices can easily be extracted
    def get_incoming(self, vertex):
        assert vertex in self.vertices, "No such vertex found"
        return [e for e in self.edges if e[1] == vertex]
        # Can get the incoming neighbors as [e[1] for e in self.get_incoming(vertex)]
        
    # Check if a vertex is a sink
    def is_sink(self, vertex):
        assert vertex in self.vertices, "No such vertex found"
        return all([e[0] != vertex for e in self.edges])

        

In [6]:
class quiver_rep:
    def __init__(self, quiver: quiver, dims: Dict, matrices: Dict):
        self.Q = quiver
        self.dims = dims
        self.matrices = matrices
        
        # Check the dimension vector
        assert len(dims) == len(Q.vertices), "Inappropriate dimension vector"
        for v in dims:
            assert v in Q.vertices, "Inappropriate dim vector"
            assert isinstance(dims[v], int) and dims[v] >=0, "Inappropriate dim vector"
            
        # Check the matrices
        assert len(matrices) == len(Q.edges), "Matrices error"
        for e in matrices:
            assert e in Q.edges, "Matrices error"
            assert isinstance(matrices[e], np.ndarray), "Matrices error" # May need fixing
            assert np.shape(matrices[e]) == (dims[e[1]], dims[e[0]]), "Dimension error"
            
            
    # Compute the reduced dimension vector
    def dims_red(self) -> Dict:
        
        assert Q.check_top_order(), "Order of the vertices is not topological"

        d_red = {}
        for i in Q.vertices:
            if Q.is_sink(i):
                d_red[i] = self.dims[i]
            else:
                incoming = Q.get_incoming(i)
                if incoming:
                    d_red[i] = min(self.dims[i], sum([d_red[e[0]] for e in incoming]) )
                else:
                    d_red[i] = self.dims[i]
                
        return d_red
        

In [8]:
# EXAMPLE
# Quiver with skip connections and no bias

vertex_list = ['a', 'b', 'c', 'd']
edge_list = [('a', 'b'), ('a','c'), ('b','c'), ('c', 'd')]

Q = quiver(vertex_list, edge_list)
# print(Q.vertices, Q.edges)

In [9]:
# Test the methods

print(Q.get_incoming('a'), Q.get_incoming('b'), Q.get_incoming('c'), Q.get_incoming('d'))
print(Q.is_sink('a'), Q.is_sink('b'), Q.is_sink('c'), Q.is_sink('d'))
print(Q.check_top_order())

[] [('a', 'b')] [('a', 'c'), ('b', 'c')] [('c', 'd')]
False False False True
True


In [10]:
# Representation of this quiver

dim_vector = {'a': 2, 'b': 4, 'c': 8, 'd': 2 }

maps = {('a', 'b') : np.random.rand(4, 2), 
        ('a', 'c') : np.random.rand(8, 2), 
        ('b', 'c') : np.random.rand(8, 4),
        ('c', 'd') : np.random.rand(2, 8)}

ex_rep = quiver_rep(Q, dim_vector, maps)
print(ex_rep.dims_red())

{'a': 2, 'b': 2, 'c': 4, 'd': 2}


In [11]:
# Troubleshooting

W1 = np.random.rand(4, 2)
print(type(W1))
print(isinstance(W1, np.ndarray))

<class 'numpy.ndarray'>
True


In [12]:
# Linear feedforward function 
# Might turn out not to be super necessary

def lin_ff(W : quiver_rep) -> np.array:
    dims = W.dims
    matrices = W.matrices
    quiver = W.Q
    vertices = quiver.vertices
    edges = quiver.edges
    
    assert quiver.check_top_order(), "Order of the vertices is not topological"
    
    # Dictionary for partial feedforward functions:
    # partial = {}
    
    for i in quiver.vertices:
        incoming = quiver.get_incoming(i)
        if incoming == []:
            # partial[i] = projection matrix
            None
        else:
            for e in incoming:
                # Add matrices[e] @ partial[e[0]]
                None
        None
    
    None

# Unfinished!

In [13]:
lin_ff(ex_rep)

# Dimensional reduction algorithm

In [59]:
def padzeros(M,newrows,newcols = None):
    oldrows, oldcols = M.shape
    if newcols == None:
        newcols = oldcols
    return np.pad(M,((0,newrows-oldrows),(0,newcols-oldcols)),mode="constant")

In [118]:
# This will eventually be incorporated in the quiver_rep class

def QRDimRed(W : quiver_rep):
    dims = W.dims
    matrices = W.matrices
    quiver = W.Q
    vertices = quiver.vertices
    edges = quiver.edges
    
    # Compute the reduced dimension vector
    dims_red = W.dims_red()
    # print(dims, dims_red)
    
    # Check that vertices are in a topological order
    assert quiver.check_top_order(), "Order of the vertices is not topological"
    
    # Q = dictionary mapping each vertex to an orthogonal matrix
    Q = {}
    
    # Vmatrices = matrices of the reduced representation V, mapping each edge to a matrix
    Vmatrices = {}
    
    print(quiver.edges)
    print(quiver.vertices)
    
    for i in vertices:
        incoming = quiver.get_incoming(i)
        
        # Case of a source vertex
        if incoming == []:
            print(i, " is a source vertex")
            Q_cur = np.eye(dims[i])
            Q[i] = Q_cur
            
        else:
            # Compute the matrices to vertex i
            M = np.array([])
            for e in incoming:
                We = matrices[e] 
                Qj = Q[e[0]]
                Me = We @ Qj[:,:dims_red[e[0]]]
                if np.shape(M) == (0,):
                    M = Me
                else:
                    M = np.hstack((M,Me))
                # Transform weights on incoming edges
                # Me = matrices[e] @ Q[e[0]] @ inc_j
                # Extend M = [M M_e]
            
            # Case of reduction
            if dims_red[i] < dims[i]: 
                print("Reduce at vertex ", i)
                Q_cur, R = np.linalg.qr(M, mode="complete")
                R = R[:dims_red[i]]
                print(np.shape(Q_cur))
                Q[i] = Q_cur
                
            # Case of no reduction
            elif not quiver.is_sink(i):
                print(i, " is not a sink")
                Q_cur, R = np.linalg.qr(M, mode="complete")
                Q[i] = Q_cur
                
            # Case of a sink
            else:
                print(i, " is a sink")
                Q_cur = np.eye(dims[i])
                Q[i] = Q_cur
                R = M
                
            # Process and add to the dictionaries
            # Q[i] = Q_cur
            for e in incoming:                       
                # Extract V_e from R_i for all incoming edges e
                Vmatrices[e] = R[:,:dims_red[e[0]]]
                R = R[:,dims_red[e[0]]:]
    
    # Make V into a representation
    V = quiver_rep(quiver, dims_red, Vmatrices)
    # print([np.shape(q) for q in Q])
    print(dims_red)
    print([np.shape(m) for m in Vmatrices.values()])
    
    
    print(Q['b'])
    print(Q['c'])
    
    # Verify that V is a subrepresentation of Q^{-1} W  
    for e in quiver.edges:
        Qi = Q[e[0]]
        Qj = Q[e[1]]
        max_diff = np.max(np.abs(np.transpose(Qj) @ matrices[e] @ Qi[:,:dims_red[e[0]]] 
                     - padzeros(Vmatrices[e], dims[e[1]])))
        assert max_diff < 1e-10
        # print("Qi has shape:", np.shape(Qi), e[0])
        # print(np.shape(np.transpose(Qj) @ matrices[e] @ Qi[:,:dims_red[e[0]]]))
        # print(np.shape(padzeros(Vmatrices[e], dims[e[1]])))
        # assert (max([np.max( np.abs(w1 - w2)) for w1, w2 in zip(QinvW, Vext)] )   )

    return
    return Q, V


In [117]:
# Back to the running example

QRDimRed(ex_rep)

[('a', 'b'), ('a', 'c'), ('b', 'c'), ('c', 'd')]
['a', 'b', 'c', 'd']
a  is a source vertex
Reduce at vertex  b
(4, 4)
Reduce at vertex  c
(8, 8)
d  is a sink
{'a': 2, 'b': 2, 'c': 4, 'd': 2}
[(2, 2), (4, 2), (4, 2), (2, 4)]
[[-0.20415483  0.13354433 -0.02854758 -0.96936668]
 [-0.26665853 -0.23216342 -0.933982    0.05168163]
 [-0.32278669 -0.89186218  0.31030866 -0.06402452]
 [-0.88488575  0.36448307  0.17484618  0.23142627]]
[[-0.10363193 -0.52977264  0.47867242  0.30695811 -0.06097888 -0.57704926
  -0.18197932 -0.12421778]
 [-0.23148968 -0.23135116 -0.30131202  0.63994432 -0.08312253  0.13979387
   0.33106808  0.50647187]
 [-0.06957588 -0.62680322 -0.29859331 -0.31780227 -0.50470766  0.31608273
  -0.19842306 -0.13457651]
 [-0.35081472 -0.1644433   0.27939318 -0.10912287  0.14861523  0.2440495
   0.70087998 -0.43248086]
 [-0.30290971 -0.25365348 -0.44048233 -0.05131933  0.74458282 -0.07174324
  -0.26181707 -0.13837778]
 [-0.47547729  0.23288105  0.31376799  0.31462416 -0.06226391  0.4

In [58]:
# Verify that V is a subrepresentation of Q^{-1} W

def verify(W : quiver_rep, V : quiver_rep, Q : Dict) -> bool:
    
    for e in quiver.edges:
        # if Q[e[1]]^{-1} @ W[e] @ Q[e[0]] @ inc_{e[0]} != inc_{e[1]} @ V[e]
        # return False
    
    None



IndentationError: expected an indented block (<ipython-input-58-526a7c20fab9>, line 9)

In [27]:
a = np.random.rand(4,2)
b = np.random.rand(4,4)
a, b

(array([[0.86051847, 0.75893567],
        [0.69781306, 0.28613325],
        [0.47965771, 0.5217031 ],
        [0.98026197, 0.17913154]]),
 array([[0.48804237, 0.94519595, 0.91748875, 0.8042133 ],
        [0.95439805, 0.67268898, 0.86469542, 0.40744969],
        [0.19802128, 0.65475544, 0.87432165, 0.50406587],
        [0.80589343, 0.55356225, 0.95174127, 0.95748327]]))

In [28]:
c = np.hstack((a,b))
np.shape(c)

(4, 6)

In [29]:
b

array([[0.48804237, 0.94519595, 0.91748875, 0.8042133 ],
       [0.95439805, 0.67268898, 0.86469542, 0.40744969],
       [0.19802128, 0.65475544, 0.87432165, 0.50406587],
       [0.80589343, 0.55356225, 0.95174127, 0.95748327]])

In [30]:
b[:,:2]

array([[0.48804237, 0.94519595],
       [0.95439805, 0.67268898],
       [0.19802128, 0.65475544],
       [0.80589343, 0.55356225]])

In [31]:
b[:,2:]

array([[0.91748875, 0.8042133 ],
       [0.86469542, 0.40744969],
       [0.87432165, 0.50406587],
       [0.95174127, 0.95748327]])

In [32]:
np.shape(b[:,:2])

(4, 2)

In [19]:
np.shape(np.array([]))

(0,)

# Quiver Neural Networks

In [10]:
class RadAct(nn.Module):
    def __init__(self, eta = F.relu):
        super().__init__()
        self.eta = eta
        self.shift = 0 
        # Add internal bias/shift later
        
    def forward(self,x):
        # x: [Batch x Channel]
        r = torch.linalg.norm(x, dim=-1) 
        if torch.min(r) < 1e-6:
            r += 1e-6
        scalar = self.eta(r + self.shift) / r
        return x * scalar.unsqueeze(-1)   

In [11]:
class QuiverNN(nn.Module):
    
    def __init__(self, eta:float , Q: quiver, dims: Dict ):
        super().__init__()
        self.eta = eta
        self.Q = Q
        sefl.dims = dims
        # Assert statement to check that dims is a dimension vector for Q
        
        # Reduced dimension vector
        # self.dims_red = [self.dims[0]]


    def forward(self, x):
        h = x
        for lin,act in zip(self.layers[:-1], self.act_fns):
            h = act(lin(h))
        return self.output_layer(h)
    
    def set_weights(self, new_weights: quiver_rep):
        None 
    
    def set_activation_biases(self, new_biases: List[float]):    
        None

    def export_weights(self) -> quiver_rep:
        None
    
    def export_activation_biases(self) -> List[float]:
        None
    
    def export_reduced_weights(self) -> quiver_rep:
        None
    
    def transformed_network(self):
        None
        
    def reduced_network(self):
        None

# Scraps

In [12]:
class vertex:
    def __init__(self):
        None

In [13]:
a = vertex()